In [1]:
import os
import shutil
import numpy as np
from ase.io import read

# === 设置路径 ===
base_dir = os.getcwd()  # 当前路径，例如 Desktop/Cu-DeePMD/00.data
traj_path = os.path.join(base_dir, "out", "Cu.traj")
out_dir = os.path.join(base_dir, "Cu_dataset", "set.000")

# === 检查轨迹文件是否存在 ===
if not os.path.exists(traj_path):
    raise FileNotFoundError(f"❌ 找不到轨迹文件: {traj_path}")

# === 读取 ASE Trajectory 文件（全部帧）===
frames = read(traj_path, index=":")
n_frames = len(frames)
print(f"✅ 成功读取 AIMD 轨迹帧数: {n_frames}")

# === 提取原子数据 ===
coords   = [atoms.get_positions() for atoms in frames]
forces   = [atoms.get_forces() for atoms in frames]
energies = [atoms.get_potential_energy() for atoms in frames]
virials  = [atoms.get_stress() * atoms.get_volume() for atoms in frames]  # 转换为真实应力张量

# === 提取原子类型信息 ===
symbols  = frames[0].get_chemical_symbols()
types    = [0 for _ in symbols]  # 全部为 Cu，编号为 0
type_map = list(set(symbols))    # 输出 ['Cu']

# === 创建输出目录（若存在则清空） ===
if os.path.exists(out_dir):
    shutil.rmtree(out_dir)
os.makedirs(out_dir)

# === 写入 raw 文件（reshape 统一为 N × D）===
np.savetxt(os.path.join(out_dir, "coord.raw"),   np.array(coords).reshape(n_frames, -1))
np.savetxt(os.path.join(out_dir, "force.raw"),   np.array(forces).reshape(n_frames, -1))
np.savetxt(os.path.join(out_dir, "energy.raw"),  np.array(energies).reshape(-1, 1))
np.savetxt(os.path.join(out_dir, "virial.raw"),  np.array(virials).reshape(n_frames, -1))
np.savetxt(os.path.join(out_dir, "type.raw"), np.array(types, dtype=int).reshape(1, -1))

# === 写入 box.raw（单位晶格为 3.5755975 立方）===
box = np.array([[3.5755975, 0, 0],
                [0, 3.5755975, 0],
                [0, 0, 3.5755975]])
boxes = np.tile(box, (n_frames, 1, 1))
np.savetxt(os.path.join(out_dir, "box.raw"), boxes.reshape(n_frames, -1))

# === 写入 type_map.raw（按行写入元素名）===
with open(os.path.join(out_dir, "type_map.raw"), "w") as f:
    f.write("\n".join(type_map) + "\n")

print(f"🎉 成功生成 DeePMD raw 数据集，保存路径: {out_dir}")


✅ 成功读取 AIMD 轨迹帧数: 500
🎉 成功生成 DeePMD raw 数据集，保存路径: /home/jovyan/Desktop/Cu-DeePMD/00.data/Cu_raw/Cu_dataset/set.000


In [2]:
# === 修复 energy / virial / type 文件的格式 ===
print("🔧 开始修复 raw 文件格式...")

# 重新获取帧数和路径
n_frames = len(coords)
raw_dir = out_dir  # 即 set.000 目录

# === 修复 energy.raw: 变成 (500, 1) ===
energy_file = os.path.join(raw_dir, "energy.raw")
energy_data = np.loadtxt(energy_file)
if energy_data.shape == (1, n_frames):
    energy_data = energy_data.T
    np.savetxt(energy_file, energy_data)
    print("✅ 已修复 energy.raw 转置为列向量")

# === 修复 virial.raw: (500, 6) → (500, 9) ===
virial_file = os.path.join(raw_dir, "virial.raw")
virial_data = np.loadtxt(virial_file)
if virial_data.shape[1] == 6:
    v6 = virial_data
    v9 = np.zeros((len(v6), 3, 3))
    v9[:, 0, 0] = v6[:, 0]  # xx
    v9[:, 1, 1] = v6[:, 1]  # yy
    v9[:, 2, 2] = v6[:, 2]  # zz
    v9[:, 1, 2] = v9[:, 2, 1] = v6[:, 3]  # yz
    v9[:, 0, 2] = v9[:, 2, 0] = v6[:, 4]  # xz
    v9[:, 0, 1] = v9[:, 1, 0] = v6[:, 5]  # xy
    np.savetxt(virial_file, v9.reshape(len(v6), -1))
    print("✅ 已修复 virial.raw 6→9 分量")

# === 修复 type.raw: (1, N) → (500, N) ===
type_file = os.path.join(raw_dir, "type.raw")
type_data = np.loadtxt(type_file, dtype=int)
if type_data.ndim == 1:
    type_data = type_data.reshape(1, -1)
if type_data.shape[0] == 1:
    type_data = np.tile(type_data, (n_frames, 1))
    np.savetxt(type_file, type_data, fmt="%d")
    print("✅ 已修复 type.raw 每帧都记录原子类型")

print("🎯 所有 raw 文件修复完成！")


🔧 开始修复 raw 文件格式...
✅ 已修复 virial.raw 6→9 分量
✅ 已修复 type.raw 每帧都记录原子类型
🎯 所有 raw 文件修复完成！


/tmp/ipykernel_1000681/3709563740.py:33: DeprecationWarning: loadtxt(): Parsing an integer via a float is deprecated.  To avoid this warning, you can:
    * make sure the original data is stored as integers.
    * use the `converters=` keyword argument.  If you only use
      NumPy 1.23 or later, `converters=float` will normally work.
    * Use `np.loadtxt(...).astype(np.int64)` parsing the file as
      floating point and then convert it.  (On all NumPy versions.)
  (Deprecated NumPy 1.23)
  type_data = np.loadtxt(type_file, dtype=int)


In [3]:
# === 修复 energy.raw 为 (500, 1) ===
import numpy as np
import os

energy_path = os.path.join(out_dir, "energy.raw")
try:
    energy_data = np.loadtxt(energy_path)

    if energy_data.ndim == 1:  # 1D 情况，修复为 2D
        energy_data = energy_data.reshape(-1, 1)
        np.savetxt(energy_path, energy_data)
        print("✅ 修复 energy.raw 为形状 (500, 1)")
    elif energy_data.shape == (500, 1):
        print("✅ energy.raw 形状已正确，无需修复")
    else:
        print(f"⚠️ energy.raw 有异常形状: {energy_data.shape}，请人工检查")

except Exception as e:
    print(f"❌ 读取 energy.raw 时出错: {e}")


✅ 修复 energy.raw 为形状 (500, 1)
